<a href="https://colab.research.google.com/github/pablocontini/Taller-de-Procesamiento-de-Datos/blob/main/TPS10_Multinomial_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enunciado

**Multinomial Naive Bayes**

La base de datos *BuzzFeed-Webis Fake News Corpus 2016* posee diferentes artículos periodísticos de una semana cercana a las elecciones estadounidenses de ese año. Se desea entrenar un algoritmo Multinomial Naive Bayes capaz de clasificar los artículos en "*mayormente falso*", "*mayormente verdadero*", "*mezcla de verdadero y falso*" y "*sin contenido factual*".

(a) *Exploración de datos*:
- Descargar la base de datos en $\texttt{https://zenodo.org/record/1239675/files/articles.
zip?download=1}$.
- Construir la base de datos. <img src="https://i.ibb.co/tTfkc8DH/image.png" width="25" />: Puede usar el siguiente código:
```python
import xml.etree.ElementTree as ET
data = {"mainText": [], "orientation": [], "veracity": []}
for filename in os.listdir("articles/"):
    root = ET.parse(f"articles/{filename}").getroot()
    for elem in root:
        if elem.tag in data.keys():
            data[elem.tag].append(elem.text)
data = pd.DataFrame(data)
data = data[data.notna().all(axis="columns")]
```
- Utilice el comando $\texttt{train_test_split}$ (sklearn) para definir dos conjuntos de datos. El
conjunto de entrenamiento debe contener el 80\% de las muestras, el resto serán de testeo.
- Utilizando $\texttt{CountVectorizer}$ (sklearn) pre-procesar los datos del texto principal de los artículos. <img src="https://i.ibb.co/tTfkc8DH/image.png" width="25" />: Se recomienda convertir el texto a minúscula, utilizar como *Stop Words* las palabras estándar del idioma inglés, eliminar palabras que aparecen en más del 60\% de los documentos y descartar las palabras vistas en menos de 3 documentos.

(b) *Entrenamiento*: Implementar un MNB de $\alpha=(1,1,\dots,1)$ que prediga la veracidad de un artículo a partir de su texto principal (pre-procesado). El código debe estar estructurado de la siguiente manera:

```python
class MNB:
    # Inicializar atributos y declarar hiperparámetros
    def __init__(self,...
    # Etapa de entrenamiento
    def fit(self,X,y):
    # Etapa de testeo soft
    def predict_proba(self,X):
    # Etapa de testeo hard (no repetir código)
    def predict(self,X):
```

(c) *Inferencia*: **Implementar** un método a la clase anterior que calcule el *accuracy* y la *Macro-F1*.

Evaluar dichas métricas en el conjunto de testeo. ¿Por qué dan tan diferentes? <img src="https://i.ibb.co/tTfkc8DH/image.png" width="25" />: Para el cálculo de la F1 debe considerar el caso de *precisión* y *recall* nulas.

(d) *Orientación*: Repetir el ejercicio, pero para clasificar la orientación política del portal donde fue publicada la noticia (izquierda, derecha o mainstream) a partir del texto principal preprocesado. ¿Siguen siendo válidas las conclusiones extraídas anteriormente? Justificar.

# (a) Exploración de datos

El dataset contiene la producción de 9 editoriales en una semana cercana a las elecciones estadounidenses. Entre las editoriales seleccionadas se encuentran 6 con un fuerte componente partidista (tres de izquierda y tres de derecha), y tres de medios tradicionales. Durante siete días laborables (del 19 al 23 de septiembre y del 26 al 27 de septiembre), periodistas profesionales de BuzzFeed verificaron cada publicación y artículo de noticias enlazado de las 9 editoriales. En total, se revisaron 1627 artículos: 826 de medios tradicionales, 256 de izquierda y 545 de derecha. El desequilibrio entre categorías se debe a las diferentes frecuencias de publicación.

Según el archivo `overview.csv`, este dataset tiene múltiples atributos adicionales al cuerpo completo del artículo:
- Metadatos del artículo:
  - `author`: autor del artículo (si está disponible).
  - `published`: fecha de publicación.
  - `title`: título del artículo.
  - `url`: enlace original de la noticia.
  - `id`: identificador único de cada artículo.
  - `publisher`: nombre del portal o medio donde fue publicado.
- `orientation`: orientación política del medio:
  - `left` (izquierda)
  - `right` (derecha)
  - `mainstream` (medios tradicionales)
- `veracity`: etiqueta de veracidad asignada por BuzzFeed:
  - `mostly true` (mayormente cierto)
  - `mostly false` (mayormente falso)
  - `mixture of true and false` (mezcla de verdadero y falso)
  - `no factual content` (sin contenido fáctico)

## Descarga y descompresión del dataset

En primer lugar, se descarga y extrae la base de datos que contiene los artículos de noticias.

In [ ]:
import requests, zipfile, io, os

# Descargar el archivo
url = "https://zenodo.org/record/1239675/files/articles.zip?download=1"
response = requests.get(url)

# Guardar en memoria y descomprimir
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.extractall()

## Construcción del DataFrame

Una vez descargados, se leen los datos de los artículos (almacenados en formato XML) y se organizan en un DataFrame de pandas. La variable `data` resultante contiene: el **texto principal**, la **orientación política** y la **etiqueta de veracidad** de cada artículo.

In [ ]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

# Inicializar el diccionario de datos
data = {"mainText": [], "orientation": [], "veracity": []}

# Iterar sobre los archivos XML
for filename in os.listdir("articles/"):
  # Parsear el archivo y obtener el elemento raíz
  root = ET.parse(f"articles/{filename}").getroot()
  # Iterar sobre los hijos directos de la raíz
  for elem in root:
      # Verificar si la etiqueta está entre las claves de interés
      if elem.tag in data.keys():
          # Agregar el texto a la lista correspondiente en el diccionario
          data[elem.tag].append(elem.text)

# Convertir a un DataFrame de pandas
data = pd.DataFrame(data)
# Eliminar filas que contengan valores nulos (NaN)
data = data[data.notna().all(axis="columns")]

## Creación de los conjuntos de entrenamiento y testeo


Antes de entrenar el modelo, se separa el conjunto original en **datos de entrenamiento** y **datos de testeo**. La partición típica de 80/20 permite que el modelo disponga de muestras variadas y reserve un subconjunto "no visto" a manera de control de calidad. Además, es conveniente **estratificar** la división según la etiqueta `veracity` para que se mantenga la proporción de clases en ambos subconjuntos, evitando sesgos; y se fija un `random_state`para garantizar reproducibilidad de los resultados en sucesivas ejecuciones.

In [ ]:
from sklearn.model_selection import train_test_split

# División de los datos en conjuntos de entrenamiento y testeo.
X_train, X_test, y_train, y_test = train_test_split(
    data["mainText"],         # features: el texto principal de los artículos
    data["veracity"],         # target: la veracidad del artículo
    test_size=0.2,            # Porcentaje de datos para el conjunto de testeo (20%)
    random_state=42,          # Semilla que garantiza la misma división en cada ejecución
    stratify=data["veracity"] # Asegura que la distribución de las clases sea la misma en ambos conjuntos
)

## Preprocesamiento con CountVectorizer

Una vez separadas las noticias, se convierten las palabras en vectores, para que el modelo pueda procesarlas.

La vectorización de un documento consiste en definir una función $f(x_1,\dots,x_n)$. El método más simple es la bolsa de palabras o Bag of Words (BoW) $f(x_1,\dots,x_n)=x_1+\dots+x_n$, donde cada coeficiente representa la **cantidad de veces que apareció** cada palabra del vocabulario.

La clase `CountVectorizer` (sklearn), hace precisamente eso, convierte una colección de documentos de texto en una matriz de recuentos de tokens.

Adicionalmente, permite aplicar normalizaciones básicas del procesamiento de lenguajes naturales (NLP):
- `lowercase=True`: unifica mayúsculas y minúsculas
- `stop_words='english'`: descarta palabras muy frecuentes o "Stop Words" (the, and, of, …) que no aportan a la clasificación.
- `max_df=0.60`: elimina términos presentes en más del 60\% de los artículos, porque su utilidad para clasificar es casi nula.
- `min_df=3`: descarta palabras demasiado raras que aparecen en ménos de 3 documentos, que pueden ser ruido tipográfico.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(
    lowercase=True,       # pasar a minúscula
    stop_words='english', # eliminar stopwords en inglés
    max_df=0.6,           # eliminar términos que aparecen en más del 60% de los documentos
    min_df=3              # eliminar términos que aparecen en menos de 3 documentos
)

# Entrenar el vectorizador
X_train_vec = cv.fit_transform(X_train)

# Transformar los datos de testeo
X_test_vec = cv.transform(X_test)

# (b) Entrenamiento de un modelo MNB

El algoritmo **Naive Bayes** es un enfoque de aprendizaje generativo utilizado para problemas de clasificación, que se basa en la modelización de la distribución condicional de las características dadas las clases, así como las probabilidades de clase a priori.

Este algoritmo recibe su nombre de la "hipótesis ingenua de Bayes" que **asume que las características de entrada son condicionalmente independientes entre sí, dado el valor de la clase**. Por ejemplo en este caso, si se clasifica un artículo como `mostly true`, la ocurrencia de una palabra como `campaign` se considera independiente de la ocurrencia de otra palabra como `debate`, una vez que se sabe que el artículo es mayormente cierto. Aunque esta suposición es extremadamente fuerte y rara vez se cumple en la realidad, el algoritmo resultante a menudo funciona sorprendentemente bien en una variedad de problemas.

En el contexto de la clasificación de texto, el algoritmo Naive Bayes puede emplear diferentes modelos de eventos. Sin embargo, para la mayoría de los problemas de clasificación de texto, el modelo de eventos multinomial (Multinomial Naive Bayes - MNB) es más adecuado y frecuentemente superior.

En este caso, se busca predecir una clase $y\in\left\{c_1,c_2,\dots,c_K\right\}$ dado un documento $x$ representado como un vector de conteo de palabras:
$$x=(x_1,x_2,\dots,x_V)$$
donde $x_i$ es la cantidad de veces que aparece la palabra $w_i$ y $V$ es el tamaño del vocabulario.

En MNB el clasificador asigna la clase según:
$$\hat{y}=\arg \max_y p(y\vert x)$$
y, por el teorema de Bayes:
$$p(y\vert x)\propto p(y)\,p(x\vert y)$$

La hipótesis Naive asume que para cada posición $j$ del documento, se selecciona una palabra según una categórica:
$$W_j\vert Y=k\sim \text{Cat}\left(p(w_1\vert y),\dots,p(w_V\vert y)\right)$$
y cada palabra del documento es una observación independiente de la misma categórica de clase $k$.

Luego, una vez generadas $N$ palabras de forma independiente de esa categórica, el número total de veces que aparece cada palabra sigue una multinomial:
$$p(x\vert y)=\frac{N!}{x_1!x_2!\dots x_V!}\prod_{i=1}^Vp(w_i\vert y)^{x_i}$$
Donde $N=\sum_{i=1}^Vx_i$ es la longitud total del documento.

El coeficiente multinomial es independiente de $y$, entonces se puede omitir para la clasificación. Por lo tanto, para el clasificador:
$$p(x\vert y)\propto \prod_{i=1}^Vp(w_i\vert y)^{x_i}$$

Y tomando el logaritmo por cuestiones de estabilidad numérica:
$$\log p(x\vert y)\propto \sum_{i=1}^V x_i \log p(w_i\vert y)$$

La estimación de parámetros se hace por máxima verosimilitud.

Para las probabilidades a priori:
$$p(y=c_k)=\frac{N_k}{N}$$
donde $N_k$ es la cantidad de documentos de la clase $c_k$ y $N$ es la cantidad total de documentos.

Para las condicionales:
$$p(w_i\vert y=c_k)=\frac{N_{ik}}{N_k^\text{total}}$$
donde $N_{ik}$ es la cantidad total de veces que aparece la palabra $w_i$ en los documentos de la clase $c_k$ y $N_k^\text{total}=\sum_{j=1}^V N_{jk}$ es el total de palabras observadas en la clase $c_k$.

Como puede haber palabras no vistas en alguna clase (lo que daría probabilidad cero), se aplica suavizado:

$$p(w_i\vert y=c_k)=\frac{N_{ik}+\alpha}{N_k^\text{total}+\alpha V}$$
donde $\alpha$ es el parámetro de suavizado.

Por lo tanto, la expresión final para la clasificación es:
$$\log p(y=c_k\vert x)\propto \log p(y=c_k)+\sum_{i=1}^V x_i \log p(w_i \vert y=c_k)$$
la cual se evalúa para cada clase $c_k$, y se elige aquella que tenga el máximo valor.

## Implementación

In [ ]:
import numpy as np

class MultinomialNB:
    """
    Clase que implementa un clasificador Multinomial Naive Bayes.

    Parameters
    ----------
    alpha : float, default=1.0
        Parámetro de suavizado. Evita probabilidades cero.
    """
    def __init__(self, alpha=1.0):
        # Inicializa el parámetro de suavizado alfa
        self.alpha = alpha
        # Atributos que se inicializarán durante el entrenamiento (fit)
        self.class_log_prior_ = None  # Logaritmo de la probabilidad a priori de cada clase
        self.feature_log_prob_ = None # Logaritmo de la probabilidad de cada característica dado cada clase
        self.classes_ = None          # Etiquetas de las clases únicas
        self.n_classes_ = None        # Número de clases
        self.n_features_ = None       # Número de características (tamaño del vocabulario)
        self.feature_count_ = None    # Recuento de cada característica por clase (antes del suavizado)
        self.class_count_ = None      # Recuento total de características por clase (antes del suavizado)


    def fit(self, X, y):
        """
        Entrena el modelo Multinomial Naive Bayes.

        Parameters
        ----------
        X : sparse matrix de forma (n_samples, n_features)
            Vectores de features de entrenamiento.
        y : array-like de forma (n_samples,)
            Etiquetas de clase para las muestras de entrenamiento.

        Returns
        -------
        self : object
            Retorna la instancia del modelo entrenado.
        """

        # Encontrar las clases únicas y sus recuentos en las etiquetas de entrenamiento
        self.classes_, counts = np.unique(y, return_counts=True)
        # Determinar el número de clases y el número de características
        self.n_classes_ = len(self.classes_)
        self.n_features_ = X.shape[1]

        # Calcular el logaritmo de la probabilidad a priori de cada clase
        # P(C) = count(C) / total_samples
        self.class_log_prior_ = np.log(counts / np.sum(counts))

        # Inicializar matrices para almacenar el recuento de features por clase
        # y el recuento total de features por clase
        self.feature_count_ = np.zeros((self.n_classes_, self.n_features_))
        self.class_count_ = np.zeros(self.n_classes_)

        # Iterar a través de cada clase para calcular los recuentos de features
        for idx, c in enumerate(self.classes_):
            # Seleccionar las filas en X que corresponden a la clase 'c'
            X_c = X[np.array(y) == c]
            # Sumar los features (columnas) para obtener el recuento total de cada feature
            # dentro de esta clase
            self.feature_count_[idx, :] = X_c.sum(axis=0)
            # Sumar todos los recuentos de features para obtener el recuento total de
            # features (palabras) en todos los documentos de esta clase
            self.class_count_[idx] = self.feature_count_[idx, :].sum()

        # Calcular el logaritmo de la probabilidad de cada feature dado cada clase
        # P(feature | class) = (count(feature, class) + alpha) / (sum(counts for class) + alpha * n_features)
        # El suavizado de Laplace (sumar alpha) evita probabilidades cero
        self.feature_log_prob_ = np.log(
            (self.feature_count_ + self.alpha) / (self.class_count_[:, np.newaxis] + self.alpha * self.n_features_)
        )

        # Retornar la instancia del objeto entrenado
        return self

    def predict_proba(self, X):
        """
        Calcula la probabilidad de cada clase para cada muestra en X.

        Parameters
        ----------
        X : sparse matrix de forma (n_samples, n_features)
            Vectores de features de entrada.

        Returns
        -------
        probs : array-like de forma (n_samples, n_classes)
            Probabilidad de cada clase para cada muestra en X.
        """

        # Calcular el logaritmo de la probabilidad no normalizada para cada clase y muestra
        # log(P(C | D)) = log(P(C)) + sum(log(P(feature | C)) por cada feature en D)
        # Donde P(C | D) es la probabilidad posterior de la clase dado el documento
        log_probs = self.class_log_prior_ + X @ self.feature_log_prob_.T

        # Normalizar los logaritmos de probabilidades para la estabilidad numérica
        # Esto evita que la exponenciación de números muy pequeños resulte en cero
        log_probs_norm = log_probs - log_probs.max(axis=1, keepdims=True)

        # Convertir los logaritmos de probabilidades normalizados a probabilidades
        probs = np.exp(log_probs_norm)

        # Normalizar las probabilidades para que sumen 1 para cada muestra
        probs = probs / probs.sum(axis=1, keepdims=True)

        return probs

    def predict(self, X):
        """
        Predice la etiqueta de clase para cada muestra en X.

        Parameters
        ----------
        X : sparse matrix de forma (n_samples, n_features)
            Vectores de features de entrada.

        Returns
        -------
        y_pred : array-like de forma (n_samples,)
            Etiquetas de clase predichas para cada muestra en X.
        """
        # Obtener las probabilidades de cada clase para cada muestra
        probs = self.predict_proba(X)
        # Encontrar el índice de la clase con la probabilidad máxima para cada muestra
        class_idx = np.argmax(probs, axis=1)
        # Retornar las etiquetas de clase correspondientes a los índices encontrados
        return self.classes_[class_idx]

# (c) Inferencia

Una creado el modelo MNB, se implementan dos métricas de evaluación utilizadas en trabajos prácticos anteriores para observar su desempeño:
- **Accuracy**: mide la proporción de artículos correctamente clasificados sobre el total.
- **Macro-F1**: promedia el F1-score calculado por clase, asignando igual peso a cada clase, independientemente de su cantidad de muestras.

| **Métrica**                    | **Expresión Matemática para su evaluación**                                                                | **Descripción**                                                                          |
| ------------------------------ | ---------------------------------------------------------------------------------------------------------------| ---------------------------------------------------------------------------------------- |
| **Accuracy**                   | $\text{Accuracy} = \frac{1}{n}\sum_{i=1}^{n} \mathbb{1}(y_i = \hat{y}_i)$ | Proporción total de muestras correctamente clasificadas.                                 |
| **Precisión (para clase $c$)** | $P_c = \frac{\mathrm{TP}_c}{\mathrm{TP}_c + \mathrm{FP}_c}$                | Porcentaje de predicciones correctas entre las predicciones positivas para la clase $c$. |
| **Recall (para clase $c$)**    | $R_c = \frac{\mathrm{TP}_c}{\mathrm{TP}_c + \mathrm{FN}_c}$                | Porcentaje de verdaderos positivos entre todos los casos reales de la clase $c$.         |
| **F1-score (para clase $c$)**  | $F1_c = \frac{2 \cdot P_c \cdot R_c}{P_c + R_c}$                           | Media armónica entre precisión y recall para la clase $c$.                               |
| **Macro-F1**                   | $\text{Macro-F1} = \frac{1}{K} \sum_{c=1}^{K} F1_c$                        | Promedio del F1-score calculado sobre todas las clases.                                  |


## Implementación de las métricas

In [ ]:
def evaluate(self, X, y_true):
    """
    Calcula accuracy y Macro-F1 sobre el conjunto X, y_true.

    Parameters
    ----------
    X : sparse matrix de forma (n_samples, n_features)
        Vectores de features para la evaluación.
    y_true : array-like de forma (n_samples,)
        Etiquetas de clase verdaderas.

    Returns
    -------
    accuracy : float
        Precisión (accuracy) del modelo.
    macro_f1 : float
        Macro-F1 del modelo.
    """

    # Realizar predicciones sobre los datos de evaluación
    y_pred = self.predict(X)

    # Calcular la precisión (accuracy)
    accuracy = np.mean(y_pred == y_true)

    # Inicializar una lista para almacenar los scores F1 por clase
    f1_scores = []
    # Iterar a través de cada clase única
    for c in self.classes_:
        # Calcular los verdaderos positivos (TP), falsos positivos (FP) y falsos negativos (FN)
        TP = np.sum((y_pred == c) & (y_true == c))
        FP = np.sum((y_pred == c) & (y_true != c))
        FN = np.sum((y_pred != c) & (y_true == c))

        # Calcular la precisión para la clase actual
        # Maneja la división por cero si TP + FP es cero
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        # Calcular el recall para la clase actual
        # Maneja la división por cero si TP + FN es cero
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0

        # Calcular el score F1 para la clase actual
        # Maneja la división por cero si precision + recall es cero
        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
        else:
            f1 = 0

        # Agregar el score F1 de la clase actual a la lista
        f1_scores.append(f1)

    # Calcular el Macro-F1 como el promedio de los scores F1 de cada clase
    macro_f1 = np.mean(f1_scores)

    return accuracy, macro_f1

# Agregar el método 'evaluate' a la clase MultinomialNB
MultinomialNB.evaluate = evaluate

## Evaluación de las métricas en el conjunto de testeo

Al evaluar las métricas de Accuracy y Macro-F1 sobre el conjunto de testeo se observa que **sus valores son muy diferentes**:
- Accuracy= 0.7227
- Macro-F1= 0.4417.

Este resultado es esperable en datasets desbalanceados como el *BuzzFeed-Webis*.

Como se puede ver en el cuadro de  conteo de cada categoría de veracidad, `mostly true` y `mixture of true and false` tienen más ejemplos, mientras que el resto tiene menos.

Entonces, si el modelo logra clasificar bien las clases mayoritarias, obtiene un **Accuracy** alto, incluso sin acertar mucho las clases minoritarias. Es decir, le **da el mismo peso a cada muestra, sin importar la clase**.

La **Macro-F1** en cambio, **le da el mismo peso a cada clase, no a cada muestra**. Por lo tanto, si hay clases que tienen pocos ejemplos y el modelo las predice mal, su valor disminuye.

Ese es el motivo por el que en problemas de clasificación desbalanceada se reporta Macro-F1 y no sólo Accuracy.

Por último, en este problema donde se evalúa la veracidad, hay **pocas palabras fuertemente indicativas de la clase**, es decir, pueden existir muchas palabras compartidas entre noticias verdaderas y falsas.

In [ ]:
# Crear una instancia de la clase MultinomialNB
mnb_veracity = MultinomialNB(alpha=1.0)
# Entrenar el modelo
mnb_veracity.fit(X_train_vec, y_train)

# Evaluar el rendimiento del modelo entrenado en el conjunto de testeo
accuracy, macro_f1 = mnb_veracity.evaluate(X_test_vec, y_test)

# Imprimir las métricas de evaluación
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro-F1: {macro_f1:.4f}")

# Mostrar el conteo de cada categoría de veracidad en el DataFrame original
data['veracity'].value_counts()

Accuracy: 0.7227
Macro-F1: 0.4417


,count
veracity,
mostly true,1249
mixture of true and false,209
mostly false,82
no factual content,64


# (d) Orientación

Una vez finalizado el análisis de la veracidad, se repite el procedimiento anterior, pero con el objetivo de predecir la orientación política del portal donde fue publicada la noticia: `left`, `right` o `mainstream`.

Al igual que en el punto anterior, se evalúan las métricas Accuracy y Macro-F1 para analizar el desempeño global. Los resultados son:
- Accuracy: 0.8037
- Macro-F1: 0.7691

Estos resultados son más consistentes que los obtenidos en el punto (c), donde había una gran diferencia entre ambas métricas.

Una de las principales razones de la mejora es que, en el caso de la orientación política, este dataset está **menos desbalanceado**. Esto se puede observar del cuadro de conteo de cada categoría de orientación.

Adicionalmente, en este caso, el texto contiene **patrones lingüísticos más indicativos** de la orientación política. Hay palabras o expresiones que son más frecuentes en portales de izquierda, derecha o mainstream.

En otras palabras, clasificar la veracidad es un problema mucho más difícil, debido al dataset desbalanceado y los patrones lingüísticos más sutiles, mientras que, para la orientación, el dataset está mejor balanceado y los patrones lingüísticos son más claros.

In [ ]:
# Seleccionar las características de entrada (texto principal) y la variable objetivo (orientación política)
X_text = data["mainText"]
y_orientation = data["orientation"]

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba.
X_train, X_test, y_train, y_test = train_test_split(
    X_text,                 # features: el texto principal de los artículos
    y_orientation,          # target: la orientación del portal
    test_size=0.2,          # Asigna el 20% de los datos al conjunto de prueba.
    random_state=42,        # Asegura que la división sea la misma cada vez para reproducibilidad.
    stratify=y_orientation  # Mantiene la misma proporción de clases de orientación en ambos conjuntos.
)

# Crear una instancia de CountVectorizer para convertir el texto en vectores de conteo de palabras
cv = CountVectorizer(
    lowercase=True,        # Convierte todo a minúsculas
    stop_words='english',  # Elimina palabras comunes en inglés
    max_df=0.6,            # Elimina términos que aparecen en más del 60% de los documentos
    min_df=3               # Elimina términos que aparecen en menos de 3 documentos
)

# Entrenar el vectorizador con los datos de entrenamiento y transformarlos
X_train_vec = cv.fit_transform(X_train)
# Transformar los datos de prueba usando el mismo vocabulario aprendido
X_test_vec = cv.transform(X_test)

# Crear una instancia del clasificador Multinomial Naive Bayes.
mnb_orientation = MultinomialNB(alpha=1.0)
# Entrenar el modelo MNB con los datos de entrenamiento vectorizados y sus etiquetas de orientación
mnb_orientation.fit(X_train_vec, y_train)

# Evaluar el modelo entrenado en el conjunto de prueba
accuracy, macro_f1 = mnb_orientation.evaluate(X_test_vec, y_test)

# Imprimir los valores de las métricas de evaluación
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro-F1: {macro_f1:.4f}")

# Mostrar el conteo de cada categoría de orientación política en el DataFrame original
data['orientation'].value_counts()

Accuracy: 0.8037
Macro-F1: 0.7691


,count
orientation,
mainstream,822
right,530
left,252
